# Bibliotecas

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import mplfinance as mplf
from sklearn.metrics import r2_score, mean_squared_error
from lazypredict.Supervised import LazyRegressor
from obterDados import obterSimbolo

# Obtendo dados

In [ ]:
rawDataM5 = obterSimbolo('WDO$')
rawDataM5

# Tratamento de Dados

### Gerando colunas com preços passados e futuros

In [ ]:
hist = rawDataM5.copy().drop(columns=['tick_volume', 'real_volume', 'spread'])
for i in range(1000):
    hist['open'+str(i+1)] = hist['open'].shift(i+1)
    hist['high'+str(i+1)] = hist['high'].shift(i+1)
    hist['low'+str(i+1)] = hist['low'].shift(i+1)
    hist['close'+str(i+1)] = hist['close'].shift(i+1)
colunas0 = ['open', 'high', 'low', 'close']
colunasY = []
for i in range(24):
    for name in colunas0:
        nomeCol = f'{name}-{str(i+1)}'
        colunasY.append(nomeCol)
        hist[nomeCol] = hist[name].shift(i-1)
hist = hist.dropna()
hist

# Modelo de Regressão

### Normalização

In [ ]:
maxVs = hist.drop(columns=colunasY).max(axis='columns')
minVs = hist.drop(columns=colunasY).min(axis='columns')
histNorm = hist.subtract(minVs, axis=0).divide(maxVs - minVs, axis=0)
histNorm

### Separar dados de treino e teste

In [ ]:
xReg = histNorm.drop(columns=colunasY)
yReg = histNorm[colunasY]
splitDays = 108 * 150
x_train_reg, x_test_reg, y_train_reg, y_test_reg = xReg.iloc[:-splitDays], xReg.iloc[-splitDays:], yReg.iloc[:-splitDays], yReg.iloc[-splitDays:]

### Lazy Regressor

In [ ]:
from lazypredict.Supervised import LazyRegressor

lazyReg = LazyRegressor()
# lazyReg.fit(x_train, x_test, y_train, y_test)

### Criando um dos modelos encontrados

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.neural_network import MLPRegressor
from math import sqrt

# modelo = OrthogonalMatchingPursuit()
modelo = MLPRegressor(verbose=True)
modelo.fit(x_train_reg, y_train_reg)
y_train_predict = modelo.predict(x_train_reg)
y_test_predict = modelo.predict(x_test_reg)
print('Treino R2:', r2_score(y_train_reg, y_train_predict))
print('Teste R2:', r2_score(y_test_reg, y_test_predict))
print('Treino RMSE:', sqrt(mean_squared_error(y_train_reg, modelo.predict(x_train_reg))))
print('Teste RMSE:', sqrt(mean_squared_error(y_test_reg, modelo.predict(x_test_reg))))


### Salvar Modelo de Regressão

In [ ]:
import pickle

pickle.dump(modelo, open('modeloRegressor.pickle', 'wb'))